In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

In [20]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = vectorizer(np.array([[s] for s in train_x])).numpy()    
    val_x = vectorizer(np.array([[s] for s in val_x])).numpy()
    test_x = vectorizer(np.array([[s] for s in test_x])).numpy()

    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = layers.Embedding(input_dim = vocab_size, 
                                       output_dim = emb_dim, 
                                       embeddings_initializer=keras.initializers.Constant(matrix),
                                       trainable=True)
    x = embedding_layer(inputs)
    x = layers.Bidirectional(layers.LSTM(32, input_dim=(None, emb_dim), 
                                         return_sequences=False, activation='tanh', 
                                         dropout=0.4))(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "lstm_models/{itr:04}".format(itr=cnt)
    checkpoint_path = model_path + "-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir $model_path
    
    model.compile('adam', "binary_crossentropy", metrics=["accuracy", 
                                                          tf.keras.metrics.AUC(name='auc')])
    
    
    #create checkpoint to save model
    #with best validation loss
    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1,
            save_weights_only=True, save_best_only=True, mode='auto')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )
    
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

mkdir: cannot create directory ‘lstm_models/0001’: No such file or directory
Epoch 1/10
63/63 [==============================] - ETA: 0s - loss: 0.6711 - accuracy: 0.5698 - auc: 0.6127
Epoch 00001: val_loss improved from inf to 0.64317, saving model to lstm_models/0001-0001.ckpt
63/63 [==============================] - 13s 212ms/step - loss: 0.6711 - accuracy: 0.5698 - auc: 0.6127 - val_loss: 0.6432 - val_accuracy: 0.6400 - val_auc: 0.6977
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 0.3891 - accuracy: 0.8444 - auc: 0.9198
Epoch 00002: val_loss improved from 0.64317 to 0.56664, saving model to lstm_models/0001-0002.ckpt
63/63 [==============================] - 12s 190ms/step - loss: 0.3891 - accuracy: 0.8444 - auc: 0.9198 - val_loss: 0.5666 - val_accuracy: 0.7440 - val_auc: 0.8344
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 0.0825 - accuracy: 0.9770 - auc: 0.9961
Epoch 00003: val_loss improved from 0.56664 to 0.55304, saving model to lstm_

In [21]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))
print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))

print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))

test avg loss:  0.49032896757125854 +/- 0.03541934508181904
test avg acc:  0.7882000088691712 +/- 0.007678546066453535
test avg auc:  0.8575724959373474 +/- 0.00932802694689113


val avg loss:  0.47632375955581663 +/- 0.04104432720115004
val avg acc:  0.801199996471405 +/- 0.013919770746109388
val avg auc:  0.8640920400619507 +/- 0.012061231470158418


train avg loss:  0.08987437039613724 +/- 0.029384184151574247
train avg acc:  0.9783499956130981 +/- 0.009930514393983347
train avg auc:  0.9977373123168946 +/- 0.0009103407375404108
